# 04. Model 2: Personalized Advice Generation

## Objective
Menghasilkan saran pembelajaran yang **personal, empatik, dan actionable** dengan menggabungkan:
- **Model 1 Output**: Cluster Label (Persona)
- **Model 3 Output**: Pace Comparison
- **Raw Data**: Exam scores, stuck points

## Approach
**Prompt Engineering with Gemini API** (Free Tier)
- ✅ No fine-tuning needed (dataset kecil, 31 users)
- ✅ Context-aware templates untuk 5 persona
- ✅ Production-ready

## 5 Persona
1. **The Sprinter**: Fast & High Score
2. **The Deep Diver**: Slow but Good Score
3. **The Night Owl**: Active 19-24 (malam)
4. **The Struggler**: Low score, banyak fail
5. **The Consistent**: Belajar rutin/stabil

In [1]:
# === 1. SETUP ===
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Paths
PROCESSED_DIR = '../data/processed'
MODELS_DIR = '../models'
os.makedirs(MODELS_DIR, exist_ok=True)

print("✅ Environment Ready!")
print(f"📁 Processed data: {PROCESSED_DIR}")
print(f"💾 Models output: {MODELS_DIR}")

✅ Environment Ready!
📁 Processed data: ../data/processed
💾 Models output: ../models


---
# STEP 1: INSTALL & CONFIGURE GEMINI API

In [2]:
# Install Gemini SDK (jika belum)
!pip install -q google-generativeai

print("✅ Gemini SDK installed!")

✅ Gemini SDK installed!


In [ ]:
import google.generativeai as genai

# ===================================
# IMPORTANT: GANTI DENGAN API KEY ANDA!
# Dapatkan gratis di: https://makersuite.google.com/app/apikey
# ===================================

API_KEY = "YOUR_API_KEY"

# Configure API
genai.configure(api_key=API_KEY)

# Test connection
model = genai.GenerativeModel('gemini-2.0-flash')
test_response = model.generate_content("Say 'Hello' in Indonesian")
print("🔗 Gemini API Connected!")
print(f"Test response: {test_response.text}")

🔗 Gemini API Connected!
Test response: Halo!


---
# STEP 2: LOAD DATA CONTEXT

File yang dibutuhkan:
1. `advice_context.csv` - Context data (nama, persona, pace)
2. `clustering_results.csv` - Cluster labels (dari Model 1)
3. `pace_features.csv` - Pace data (dari Model 3)

In [10]:
print("="*60)
print("LOADING CONTEXT DATA")
print("="*60)

# Load main context file
df_advice = pd.read_csv(os.path.join(PROCESSED_DIR, 'advice_context.csv'))
print(f"\n📂 Advice context loaded: {df_advice.shape}")
print(f"Columns: {list(df_advice.columns)}")

# Load clustering results (untuk cluster_label)
df_clusters = pd.read_csv(os.path.join(PROCESSED_DIR, 'clustering_results.csv'))
print(f"\n📂 Clustering results loaded: {df_clusters.shape}")

# Merge cluster labels jika belum ada di advice_context
if 'cluster_label' not in df_advice.columns:
    df_advice = df_advice.merge(
        df_clusters[['developer_id', 'journey_id', 'cluster_label']], 
        on=['developer_id', 'journey_id'], 
        how='left'
    )
    print("✅ Cluster labels merged!")

print(f"\n📊 Final dataset shape: {df_advice.shape}")
print(f"✅ Ready for advice generation!")

LOADING CONTEXT DATA

📂 Advice context loaded: (2008, 18)
Columns: ['developer_id', 'journey_id', 'name', 'avg_study_hour', 'study_time_slot', 'avg_exam_score', 'exam_fail_count', 'avg_submission_rating', 'submission_fail_count', 'completion_speed', 'performance_level', 'struggle_score', 'speed_category', 'id', 'display_name', 'stuck_tutorial_id', 'pace_insight', 'cluster_label']

📂 Clustering results loaded: (1907, 9)

📊 Final dataset shape: (2008, 18)
✅ Ready for advice generation!


In [11]:
# Preview data
print("\n🔍 Sample data:")
display(df_advice[[
    'developer_id', 'journey_id', 'name', 'cluster_label', 
    'avg_exam_score', 'completion_speed'
]].head())

print("\n📊 Cluster distribution:")
print(df_advice['cluster_label'].value_counts())


🔍 Sample data:


,developer_id,journey_id,name,cluster_label,avg_exam_score,completion_speed
0,3390,14,Belajar Fundamental Aplikasi Android,The Struggler,62.500000,0.521429
1,3390,32,Belajar Membangun LINE Chatbot,The Consistent,83.846154,0.560000
2,3390,47,Menjadi Game Developer Expert,The Struggler,83.846154,0.578571
3,3390,51,Belajar Membuat Aplikasi Android untuk Pemula,The Consistent,65.000000,0.560000
4,3390,55,Kotlin Android Developer Expert,The Struggler,83.846154,0.600000



📊 Cluster distribution:
cluster_label
The Sprinter      762
The Struggler     528
The Night Owl     375
The Consistent    343
Name: count, dtype: int64


---
# STEP 3: PROMPT ENGINEERING

## Design Strategy:
1. **Base Template**: Struktur umum untuk semua persona
2. **Persona-Specific Context**: Variasi saran berdasarkan cluster
3. **Dynamic Variables**: Name, score, pace, stuck_point

In [12]:
# === PROMPT TEMPLATES ===

# Base template (untuk semua persona)
BASE_TEMPLATE = """
Kamu adalah AI Learning Coach untuk platform pembelajaran programming Dicoding.

DATA SISWA:
- Nama: {name}
- Persona: {persona}
- Kecepatan Belajar: {pace_info}
- Rata-rata Score Kuis: {avg_score:.1f}
- Status: {status_info}

TUGAS:
Berikan saran pembelajaran yang:
1. **Personal**: Panggil nama siswa
2. **Empatik**: Sesuai dengan persona dan situasi mereka
3. **Actionable**: Berikan 1-2 saran konkret yang bisa langsung dipraktikkan

{persona_context}

FORMAT OUTPUT:
- 1-2 paragraf dalam Bahasa Indonesia
- Tone: Sopan, memotivasi, dan suportif
- Hindari jargon teknis yang rumit
"""

# Persona-specific context
PERSONA_CONTEXTS = {
    'The Sprinter': """
CONTEXT PERSONA:
Siswa ini adalah Fast Learner yang menyelesaikan materi dengan cepat dan nilai tinggi.
FOKUS SARAN: 
- Apresiasi kecepatan dan kemampuannya
- Sarankan tantangan lebih tinggi (advanced topics, project-based learning)
- Dorong sharing knowledge dengan teman
""",
    
    'The Deep Diver': """
CONTEXT PERSONA:
Siswa ini belajar dengan tempo lambat tapi mendalam (reflective learner).
FOKUS SARAN:
- Apresiasi ketelitian dan pemahaman mendalamnya
- Sarankan tetap pertahankan kualitas, jangan terburu-buru
- Dorong dokumentasi proses belajarnya
""",
    
    'The Night Owl': """
CONTEXT PERSONA:
Siswa ini aktif belajar di malam hari (jam 19.00 - 24.00).
FOKUS SARAN:
- Akui jadwal belajar malamnya
- Sarankan optimasi jadwal (breaks, kesehatan mata)
- Ingatkan pentingnya istirahat cukup
""",
    
    'The Struggler': """
CONTEXT PERSONA:
Siswa ini mengalami kesulitan (nilai rendah, banyak submission/kuis gagal).
FOKUS SARAN:
- Berikan motivasi dan empati
- Sarankan resources tambahan (review materi, forum diskusi)
- Dorong untuk tidak menyerah, satu langkah kecil dulu
""",
    
    'The Consistent': """
CONTEXT PERSONA:
Siswa ini belajar dengan rutin dan konsisten setiap hari/minggu.
FOKUS SARAN:
- Apresiasi konsistensi dan disiplinnya
- Sarankan tetap pertahankan momentum
- Dorong set target jangka panjang
"""
}

print("✅ Prompt templates defined!")
print(f"📝 Base template length: {len(BASE_TEMPLATE)} chars")
print(f"🎭 Personas covered: {list(PERSONA_CONTEXTS.keys())}")

✅ Prompt templates defined!
📝 Base template length: 586 chars
🎭 Personas covered: ['The Sprinter', 'The Deep Diver', 'The Night Owl', 'The Struggler', 'The Consistent']


---
# STEP 4: ADVICE GENERATION FUNCTION

In [13]:
def generate_advice(row, model, verbose=False):
    """
    Generate personalized advice untuk satu user-journey pair.
    
    Args:
        row: DataFrame row dengan info user
        model: Gemini model instance
        verbose: Print progress atau tidak
    
    Returns:
        str: Generated advice text
    """
    try:
        # Extract data
        name = row.get('display_name', 'Siswa')  
        course_name = row.get('name', 'Kursus')  
        persona = row.get('cluster_label', 'Unknown')
        avg_score = row.get('avg_exam_score', 0)
        completion_speed = row.get('completion_speed', 1.0)
        
        # Build pace info
        if completion_speed < 0.7:
            pace_info = f"Lebih cepat {(1-completion_speed)*100:.0f}% dari rata-rata"
        elif completion_speed > 1.3:
            pace_info = f"Lebih lambat {(completion_speed-1)*100:.0f}% dari rata-rata"
        else:
            pace_info = "Sesuai dengan rata-rata kelas"
        
        # Build status info
        if avg_score >= 85:
            status_info = "Performa sangat baik"
        elif avg_score >= 70:
            status_info = "Performa cukup baik"
        else:
            status_info = "Perlu peningkatan"
        
        # Get persona context
        persona_context = PERSONA_CONTEXTS.get(persona, "")
        
        # Build final prompt
        prompt = BASE_TEMPLATE.format(
            name=name,
            persona=persona,
            pace_info=pace_info,
            avg_score=avg_score,
            status_info=status_info,
            persona_context=persona_context
        )
        
        if verbose:
            print(f"\n🤖 Generating advice for: {name} - {course_name} ({persona})...")
        
        # Call Gemini API
        response = model.generate_content(prompt)
        advice_text = response.text.strip()
        
        if verbose:
            print(f"✅ Generated {len(advice_text)} chars")
        
        return advice_text
        
    except Exception as e:
        print(f"❌ Error for {row.get('developer_id', 'unknown')}: {str(e)}")
        return f"[Error generating advice: {str(e)}]"

print("✅ Advice generation function ready!")

✅ Advice generation function ready!


---
# STEP 5: TEST GENERATION (SAMPLE)

Test dengan 3 sample users (berbeda persona) sebelum batch generation.

In [ ]:
print("="*60)
print("TESTING ADVICE GENERATION (3 SAMPLES)")
print("="*60)

# Pilih 3 sample dari persona berbeda
sample_personas = df_advice['cluster_label'].unique()[:3]
samples = []

for persona in sample_personas:
    sample = df_advice[df_advice['cluster_label'] == persona].iloc[0]
    samples.append(sample)

print(f"\n📋 Testing {len(samples)} samples...\n")

# Generate untuk setiap sample
for idx, sample in enumerate(samples, 1):
    print(f"\n{'='*60}")
    print(f"SAMPLE {idx}: {sample['cluster_label']}")
    print(f"{'='*60}")
    print(f"Name: {sample.get('display_name', 'N/A')}")
    print(f"Score: {sample.get('avg_exam_score', 0):.1f}")
    print(f"Speed: {sample.get('completion_speed', 1.0):.2f}x")
    
    advice = generate_advice(sample, model, verbose=True)
    
    print(f"\n📝 GENERATED ADVICE:\n")
    print(advice)
    print(f"\n(Length: {len(advice)} chars)")
    
    # Rate limiting (to avoid API quota)
    time.sleep(2)

print(f"\n\n✅ Test generation completed!")

TESTING ADVICE GENERATION (3 SAMPLES)

📋 Testing 3 samples...


SAMPLE 1: The Struggler
Name: Belajar Fundamental Aplikasi Android
Score: 62.5
Speed: 0.52x

🤖 Generating advice for: istiabudi73 - Belajar Fundamental Aplikasi Android (The Struggler)...
✅ Generated 1166 chars

📝 GENERATED ADVICE:

Halo istiabudi73, saya tahu kadang perjalanan belajar pemrograman bisa terasa menantang dan wajar jika ada saatnya kita merasa kesulitan. Tapi jangan khawatir, ini adalah bagian alami dari proses belajar! Apalagi, kamu punya kecepatan belajar yang di atas rata-rata, lho! Itu adalah modal yang luar biasa untuk bisa mengejar ketertinggalan dan memahami materi lebih dalam.

Untuk membantu kamu meningkatkan pemahaman dan skor kuis, saya punya dua saran konkret. Pertama, coba luangkan waktu sebentar untuk meninjau kembali materi yang baru kamu pelajari, terutama pada bagian yang terasa paling sulit atau di mana kamu sering mendapatkan nilai rendah. Kamu bisa mencoba membuat catatan kecil atau menjel